In [2]:
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np 
import cv2
import os
import csv
import xgboost as xgb
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV

In [21]:
def read_model():
    #url="https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
    url="https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/classification/5"
    #url="https://tfhub.dev/tensorflow/efficientnet/b0/classification/1"
    base_model = hub.KerasLayer(url, input_shape=(200,200,3))

    #base_model.trainable = False #Freezing layers
    model = keras.Sequential([
        base_model,
        #layers.Dense(128,activation='relu'),
        #layers.Dense(4, activation='softmax')
    ])
    '''
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    model.fit(x,y, epochs=15, verbose=2)
    '''
    return model

model=read_model()

In [8]:
df=pd.read_csv('../recognition/ids.csv', names=['image','class'], header=None)

def read_images_labels(directory,traintest):
    images=[]
    labels=[]
    for f in os.listdir(directory):                                 
        img=cv2.imread(directory+"/"+f , cv2.IMREAD_COLOR)
        img_resized = cv2.resize(img, (200, 200), interpolation = cv2.INTER_LINEAR)
        images.append(img_resized)   
        #print(img)
        #cv2.imshow('image',img)
        #cv2.waitKey(0)
        labels.append(list(df[df.image==traintest+'/'+f]['class'])[0])
    return images, labels

In [9]:
images_train, images_train_labels =read_images_labels('../../dataset_cropped/train_cropped','train')
images_test, images_test_labels=read_images_labels('../../dataset_cropped/test_cropped','test')        

In [10]:
images_train=np.array(list(images_train))/255
images_test=np.array(list(images_test))/255

In [22]:
features_train=(model.predict(images_train))
features_test=(model.predict(images_test))

In [23]:
xg_model = xgb.XGBClassifier(_label_encoder=False)
xg_model.fit(features_train, images_train_labels)
y_predicted = xg_model.predict(features_test)

print("Overall CA", accuracy_score(y_predicted, images_test_labels))
#print(classification_report(images_test_labels, y_predicted))


C:\Users\marko\anaconda3\envs\tensorflow\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:20:53] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "_label_encoder" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:20:54] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Overall CA 0.07228915662650602


In [24]:
model = LogisticRegression(
    solver='liblinear',
    C=1, #inverse of regularization strength (bigger c bigger regularization)
    penalty='l2',
    max_iter=100, #max num of iterations performed by solver 
    random_state=0).fit(features_train,images_train_labels)

#print(model.coef_) #to get the coeficients
y_predicted = model.predict(features_test)
print("Overall CA", accuracy_score(y_predicted, images_test_labels))
#print(classification_report(images_test_labels, y_predicted))
#len(model.coef_[0])

Overall CA 0.24096385542168675
